In [4]:
import pickle
import torch

In [94]:
cov_path = "./class_barycenter/COCO_train/1_cov.pkl"
mean_path = "./class_barycenter/COCO_train/1_mean.pkl"

with open(cov_path, "rb") as f:
    covariance1 = pickle.load(f)

with open(mean_path, "rb") as f:
    mean1 = pickle.load(f)

In [95]:
cov_path = "./class_barycenter/COCO_train/2_cov.pkl"
mean_path = "./class_barycenter/COCO_train/2_mean.pkl"

with open(cov_path, "rb") as f:
    covariance2 = pickle.load(f)

with open(mean_path, "rb") as f:
    mean2 = pickle.load(f)

In [137]:
def KL_divergence(mean1, mean2, covariance1, covariance2):
    assert covariance1.shape == covariance2.shape
    d = covariance1.shape[0]
    sign = [-1, 1]
    lndet = torch.linalg.slogdet(covariance2 @ torch.linalg.inv(covariance1))
    lndet = lndet.logabsdet * sign[int(lndet.sign)]
    prod = covariance2 @ torch.linalg.inv(covariance1)
    scal_prod = torch.t(mean2-mean1) @ torch.linalg.inv(covariance1) @ (mean2-mean1)
    return 1 / 2 * lndet - d / 2 + 1 / 2 * torch.trace(prod) + 1 / 2 * scal_prod

In [143]:
KL_divergence(mean1, mean1, covariance1, covariance1)

tensor(-3.0518e-05, device='cuda:0')

In [124]:
import os
path = "./class_barycenter/COCO_train"


mean = [pkl for pkl in os.listdir("./class_barycenter/COCO_train") if pkl.split('_')[1] == 'mean.pkl']
cov =  [pkl for pkl in os.listdir("./class_barycenter/COCO_train") if pkl.split('_')[1] == 'cov.pkl']

In [122]:
import numpy as np

n = len(mean)
heatmap = np.zeros((n, n))

In [145]:
mean1_path

'./class_barycenter/COCO_train\\9_mean.pkl'

In [139]:
for i in range(n):
    mean1_path = os.path.join(path, mean[i])
    cov1_path = os.path.join(path, cov[i])
    with open(mean1_path, 'rb') as f:
        mean1 = pickle.load(f)
    with open(cov1_path, 'rb') as f:
        covariance1 = pickle.load(f)
    for j in range(n):
        mean2_path = os.path.join(path, mean[j])
        cov2_path = os.path.join(path, cov[j])
        with open(mean2_path, 'rb') as f:
            mean2 = pickle.load(f)
        with open(cov2_path, 'rb') as f:
            covariance2 = pickle.load(f)

        heatmap[i, j] = KL_divergence(mean1, mean2, covariance1, covariance2)

In [133]:
heatmap

array([[1718.57958984, 1261.83972168, 1898.01513672, ..., 1494.79589844,
        1744.18188477, 2389.68017578],
       [2580.91992188, 1718.5793457 , 3225.25537109, ..., 2573.23681641,
        3896.42919922, 3830.80322266],
       [2997.51708984, 2193.47045898, 1718.57946777, ..., 2584.38378906,
        6578.38867188, 2772.10180664],
       ...,
       [2063.70043945, 1602.87463379, 2017.95458984, ..., 1718.57958984,
        2080.04125977, 2773.20629883],
       [1845.25012207, 1377.19506836, 2099.55786133, ..., 1551.64050293,
        1718.57958984, 2721.92041016],
       [1676.65930176, 1065.30932617, 1242.9083252 , ..., 1304.16186523,
        1833.86657715, 1718.57958984]])

In [140]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt

plt.imshow(heatmap)